## Setting up

# Arduino Nano BLE33 Sense

This is a [micro-controller board](./ABX00031-datasheet.pdf) that has a few sensors on it. I will be specifically looking at 6 channels of the IMU, (3 acceleration channels and the 3 magnetic field channels). The acceleration channels will be used to figure the angle of the Optical Tube Assembly (OTA), commonly refereed to as the altitude (ALT).  The 3 magnetic field channels will be used to determine the direction the OTA is pointed, commonly refereed to as the azimuth (AZ). 

I want to see how accurate they are.  I do know that the ALT/AZ can be identified by the stars (image plate), time of day and location. If the IMU can be used to some degree we mat be able to get some basic functionality without having to process the image plates.

The BLE33 Sense can also be used to run a machine learning model. It may be feasible if the accuracy is good enough to determine the Declination the scope is pointed at and to do star tracking based on that.  


In [2]:
!python3 -m pip install pyserial

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import serial.tools.list_ports

ports = serial.tools.list_ports.comports()

print([port.name for port in ports])

['ttyACM0']


In [4]:
sensors = serial.Serial('/dev/ttyACM0')

In [5]:
import threading
#values
#['aX', 'aY', 'aZ', 'mX', 'mY', 'mZ\r\n']

accelerometer = {}
magnetometer = {}

def read_sensors():
    global accelerometer
    global magnetometer

    while True:
        str_values = sensors.read_until(expected=b"\r\n").decode().split("\t")
        if len(str_values) == 6:
            accelerometer = {'aX' : float(str_values[0]), 'aZ' : float(str_values[1]), 'aY' : float(str_values[2])}
            magnetometer = {'mX' : float(str_values[3]), 'mZ' : float(str_values[4]), 'mY' : float(str_values[5])}

threading.Thread(target=read_sensors).start()
#str_values
#values = line.split("\t")

#print(values)
#print(motors.read_until(expected=b"\r\n").decode())

In [6]:
accelerometer, magnetometer

({'aX': -0.03, 'aZ': -1.01, 'aY': -0.03},
 {'mX': -4.96, 'mY': 36.77, 'mZ': 30.29})

In [5]:
#({'aX': 0.99, 'aY': -0.06, 'aZ': -0.1}, {'mX': 40.0, 'mY': 4.65, 'mZ': 31.58})
#({'aX': 0.99, 'aY': -0.06, 'aZ': -0.1}, {'mX': 41.24, 'mY': -7.71, 'mZ': 29.0})
hold = accelerometer
hold

{'aZ': 0.99, 'aX': -0.04, 'aY': -0.1}

## Calculate the tilt with respect to _x_ (ALT)

From this [post](https://www.digikey.com/en/articles/using-an-accelerometer-for-inclination-sensing).

You need to do this because the accelerometor is sinasoidal in nature.

In [80]:
import math 
print(accelerometer)
tiltX = math.atan(accelerometer['aX']/math.sqrt((accelerometer['aY']*accelerometer['aY']) + (accelerometer['aZ']*accelerometer['aZ'])))*(180/math.pi)
tiltX               

{'aX': 0.99, 'aZ': -0.04, 'aY': -0.11}


83.25726133339383

There is offset error do to a few things; like the non-perfect attachment of the sensor. It has a slight angle. In the 3 axis. So I measured calculated angles when I knew the angle based on putting a level on the scope. I then used the 2 points to linearize the offset to be applied. Thus making the angle fit better.

In [78]:
calculated90 = 83.04141110222139
calculated0 = -1.7006050180649772

#y=mx+b y= angle  and x= offset
m= (90-0)/(calculated90 - calculated0)
b= 0 - (m * calculated0)

m,b

84.74201612028637


(1.0620469528627952, 1.8061223774590878)

The following cell reports the Altitude for ALT/AZ So we can con vert to RA/DEC

In [81]:
print(accelerometer)
m*(math.atan(accelerometer['aX']/math.sqrt((accelerometer['aY']*accelerometer['aY']) + (accelerometer['aZ']*accelerometer['aZ'])))*(180/math.pi)) + b

{'aX': 0.99, 'aZ': -0.04, 'aY': -0.11}


90.22924308029143

## Determine Compas Heading (AZmuth)

https://www.w3.org/TR/magnetometer/

In [273]:

def calibrate_heading( expected_heading : float ) -> float :
    calculated_heading = math.atan2(magnetometer['mY'],magnetometer['mX']) * (180 / math.pi)
    return expected_heading - calculated_heading                      

offset = calibrate_heading(280)


In [274]:
int((math.atan2(magnetometer['mZ'], magnetometer['mY']) * (180 / math.pi)) + offset)

279